In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pickle
import pandas as pd
import numpy as np

In [17]:
####load the trained model,scaler pickle ,onehot encoder pickle
model = load_model('model.h5')
##load the encoder and scaler

with open('label_encoder_gender.pkl', 'rb') as f:
    label_encoder = pickle.load(f)

with open('onehot_encoder.pkl', 'rb') as f:
    onehot_encoder = pickle.load(f)
    
with open('scaler.pkl', 'rb') as f:
    scaler = pickle.load(f)

In [18]:
#example input data
input_data = {
    'CreditScore': 700,
    'Geography': 'France',
    'Gender':'Male',
    'Age': 40,
    'Tenure': 3,
    'Balance': 60000,
    'NumOfProducts': 2,
    'HasCrCard': 1,
    'IsActiveMember': 1,     
    'EstimatedSalary': 50000
}   

In [19]:
geo_encoded = onehot_encoder.transform([[input_data['Geography']]]).toarray()
geo_encoded_df = pd.DataFrame(geo_encoded,columns=onehot_encoder.get_feature_names_out(['Geography']))
geo_encoded_df 

c:\Users\303370\Desktop\ann_learning\venv\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0


In [20]:
# Create DataFrame from input dictionary and drop Geography before concatenating
input_df = pd.DataFrame([input_data])
input_data_processed = input_df.drop('Geography', axis=1)
input_data_processed = pd.concat([input_data_processed.reset_index(drop=True), geo_encoded_df], axis=1)
print(type(input_data_processed))
input_data_processed

<class 'pandas.core.frame.DataFrame'>


,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,700,Male,40,3,60000,2,1,1,50000,1.0,0.0,0.0


In [21]:
#encode categorical variables
input_data_processed['Gender'] = label_encoder.transform(input_data_processed['Gender'])
input_data_processed
input_scaled = scaler.transform(input_data_processed)
input_scaled

array([[ 0.50178794,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221,  1.00150113,
        -0.57946723, -0.57638802]])

In [22]:
##PREDICT CHURN
prediction = model.predict(input_scaled)
prediction

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step


array([[0.0161876]], dtype=float32)

In [ ]:
prediction_proba = prediction[0][0]
prediction_proba = prediction_proba * 100
print(f"Predicted probability of churn: {prediction_proba:.2f}%")

np.float32(0.016187599)

In [24]:
if prediction_proba > 50:
    print("The customer is likely to churn.")   
else:
    print("The customer is unlikely to churn.")

The customer is unlikely to churn.
